In [ ]:
from math import sqrt
from random import randint,shuffle
import random
import time

# Defining infinity
INF = 1e18
mult = 1e6
 
# Structure to represent a 2D point
class Point :
    def __init__(self,X=0,Y=0) -> None:
        self.X=X
        self.Y=Y
        
# Structure to represent a 2D circle
class Circle :
    def __init__(self,c=Point(),r=0) -> None:       
        self.C=c
        self.R=r
 
 
# Function to return the euclidean distance
# between two points
def dist(a, b):
    return sqrt(pow(a.X - b.X, 2)
                + pow(a.Y - b.Y, 2))

# Function to check whether a point lies inside
# or on the boundaries of the circle
def is_inside(c, p):
    return dist(c.C, p) <= c.R
 
# The following two functions are used
# To find the equation of the circle when
# three points are given.
 
# Helper method to get a circle defined by 3 points
def get_circle_center(bx, by,
                        cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D),
             (bx * C - cx * B) / (2 * D))
 
# Function to return the smallest circle
# that intersects 2 points
def circle_from1(A, B):
    # Set the center to be the midpoint of A and B
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0 )
 
    # Set the radius to be half the distance AB
    return Circle(C, dist(A, B) / 2.0 )
 
# Function to return a unique circle that
# intersects three points
def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y,
                                C.X - A.X, C.Y - A.Y)
 
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))
 
 
 
 
 
# Function to check whether a circle
# encloses the given points
def is_valid_circle(c, P):
 
    # Iterating through all the points
    # to check  whether the points
    # lie inside the circle or not
    for p in P:
        if (not is_inside(c, p)):
            return False
    return True
 
# Function to return the minimum enclosing
# circle for N <= 3
def min_circle_trivial(P):
    assert(len(P) <= 3)
    if not P :
        return Circle()
     
    elif (len(P) == 1) :
        return Circle(P[0], 0)
     
    elif (len(P) == 2) :
        return circle_from1(P[0], P[1])
     
 
    # To check if MEC can be determined
    # by 2 points only
    for i in range(3):
        for j in range(i + 1,3):
 
            c = circle_from1(P[i], P[j])
            if (is_valid_circle(c, P)):
                return c
         
     
    return circle_from2(P[0], P[1], P[2])
 
# Returns the MEC using Welzl's algorithm
# Takes a set of input points P and a set R
# points on the circle boundary.
# n represents the number of points in P
# that are not yet processed.
def welzl_helper(P, R, n):
    # Base case when all points processed or |R| = 3
    if (n == 0 or len(R) == 3) :
        return min_circle_trivial(R)
     
 
    # Pick a random point randomly
    idx = randint(0,n-1)
    p = P[idx]
 
    # Put the picked point at the end of P
    # since it's more efficient than
    # deleting from the middle of the vector
    P[idx], P[n - 1]=P[n-1],P[idx]
 
    # Get the MEC circle d from the
    # set of points P - :p
    d = welzl_helper(P, R.copy(), n - 1)
 
    # If d contains p, return d
    if (is_inside(d, p)) :
        return d
     
 
    # Otherwise, must be on the boundary of the MEC
    R.append(p)
 
    # Return the MEC for P - :p and R U :p
    return welzl_helper(P, R.copy(), n - 1)

def minimum_enclosing_circle(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))

if __name__=='__main__':
    alt = 1000                     
    lat_position = 24.788090       
    lon_position = 120.997858      
    TxR = 300
    lat = [0]*500
    lon = [0]*500
    laspeed = [0]*500
    lospeed = [0]*500
    add_flag = 0
    lospeed_total = 0
    laspeed_total = 0
    latmin = 24.7890
    latmax = 24.7895
    lonmin = 120.9975
    lonmax = 120.9980
    smax = 200*0.000009
    smin = 20*0.000009
    user = 100
    count_out = 0
    count_num = 0
    time_max = 0
    ladiff_total = 0
    lodiff_total = 0
    
    
    
    f= open("users_6G2.txt","r")
    while True:
        laspeed_total = 0
        lospeed_total = 0
        count_num += 1
        start = time.time()
        print("Current users number = ",user)
        for i in range(100):
            line = f.readline()
            lat[i] = float(line)
            line = f.readline()
            lon[i] = float(line)
            line = f.readline()
            laspeed[i] = float(line)
            line = f.readline()
            lospeed[i] = float(line)
            
            
        v1 = [Point(lon[0]*mult,lat[0]*mult)]
        for i in range(1, user):
            v1.append(Point(lon[i]*mult,lat[i]*mult))
        mec = minimum_enclosing_circle(v1)
        
        for i in range(0,user):
            lospeed_total += lospeed[i];
            laspeed_total += laspeed[i];
        lospeed_uav = lospeed_total/user;
        laspeed_uav = laspeed_total/user;

        
        if mec.R/mult > TxR*0.000009:
            print("Out of range")
            for i in range(user-1 , -1, -1): # 99~0
                v2 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                for j in range(user-2 , i-1, -1): #user-i
                    v2.append(Point(lon[j]*mult,lat[j]*mult))
                mec2 = minimum_enclosing_circle(v2);
                if mec2.R/mult > TxR*0.000009:
                    print("Kicked off user below ",i)
                    v3 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                    for j in range(user-2 , i ,-1):
                        v3.append(Point(lon[j]*mult,lat[j]*mult))
                    mec3 = minimum_enclosing_circle(v3)
                    for m in range(i+1):
                        if sqrt(pow(lat[m] - mec3.C.Y/mult, 2) + pow(lon[m] - mec3.C.X/mult, 2)) > TxR * 0.000009:
                            print("Kick off user ",m)
                            lat[m] = 0
                    cnt = 0
                    for k in range(user):
                        if lat[k] != 0:
                            #print("lat[",cnt,"] = lat[",k,"]")
                            lat[cnt] = lat[k]
                            cnt += 1
                    user = cnt
                    if mec3.C.Y/mult > lat_position:
                        print("前移",(mec3.C.Y/mult - lat_position) / 0.000009,"公尺")
                    else:
                        print("後移",(lat_position - mec3.C.Y/mult) / 0.000009,"公尺")
                    if mec3.C.X/mult > lon_position:
                        print("左移",(mec3.C.X/mult - lon_position) / 0.000009,"公尺")
                    else:
                        print("右移",(lon_position - mec3.C.X/mult) / 0.000009,"公尺")
                    lat_position = mec3.C.Y/mult
                    lon_position = mec3.C.X/mult
                    break
        elif mec.R/mult == 0:
            print("mec.Y = %5.6f" %mec.C.Y/mult)
            print("mec.X = %5.6f" %mec.C.X/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
           
        else:
            print("lat = ",mec.C.Y/mult)
            print("lon = ",mec.C.X/mult)
            #print("R = ",mec.R/mult)
            if mec.C.Y > lat_position:
                print("前移",(mec.C.Y - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y) / 0.000009,"公尺")
            if mec.C.X > lon_position:
                print("左移",(mec.C.X - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X) / 0.000009,"公尺")
            print("real laspeed = ",abs(lat_position - mec.C.Y/mult) / 0.000009/0.03,"m/s")
            print("real lospeed = ",abs(lon_position - mec.C.X/mult) / 0.000009/0.03,"m/s")
            print("predict laspeed = ",laspeed_uav/0.03,"m/s")
            print("predict lospeed = ",lospeed_uav/0.03,"m/s")
            larspd = abs((lat_position - mec.C.Y/mult) / 0.000009/0.03)
            lorspd = abs((lon_position - mec.C.X/mult) / 0.000009/0.03)
            lapspd = abs(laspeed_uav/0.03)
            lopspd = abs(lospeed_uav/0.03)
            ladiff = abs((larspd - lapspd)/larspd)
            lodiff = abs((lorspd - lopspd)/lorspd)
            
            if count_num > 1:
                ladiff_total = ladiff_total + ladiff
                lodiff_total = lodiff_total + lodiff
                ladiff_mean = ladiff_total/(count_num-1)
                lodiff_mean = lodiff_total/(count_num-1)
                print("la誤差率 = ", ladiff_mean)
                print("lo誤差率 = ", lodiff_mean)
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
            
        end = time.time()
        t = end - start
        if t < 0.03:
            time.sleep(0.03-t)

Current users number =  100
lat =  24.791924285201716
lon =  120.99971968032227
前移 2754655499679.0796 公尺
左移 13444399853607.14 公尺
real laspeed =  14201.056302651536 m/s
real lospeed =  6895.112304734826 m/s
predict laspeed =  19.932724333333333 m/s
predict lospeed =  19.388606333333332 m/s
finish
Current users number =  100
lat =  24.79192969840884
lon =  120.99972607931159
前移 2754656100720.506 公尺
左移 13444400564399.1 公尺
real laspeed =  20.04891527199696 m/s
real lospeed =  23.69996042051315 m/s
predict laspeed =  19.964893666666665 m/s
predict lospeed =  19.460593666666668 m/s
la誤差率 =  0.004190830485859352
lo誤差率 =  0.17887653306699877
finish
Current users number =  100
lat =  24.791934345176664
lon =  120.99973216236585
前移 2754656617027.4404 公尺
左移 13444401240293.307 公尺
real laspeed =  17.2102511994707 m/s
real lospeed =  22.529830590445744 m/s
predict laspeed =  19.93818933333333 m/s
predict lospeed =  19.32338933333333 m/s
la誤差率 =  0.08134870743039428
lo誤差率 =  0.16059817260203796
finis

Current users number =  100
lat =  24.79203990425494
lon =  120.99987252029474
前移 2754668345802.1836 公尺
左移 13444416835603.143 公尺
real laspeed =  17.103650130757664 m/s
real lospeed =  22.473958141391552 m/s
predict laspeed =  19.864029333333335 m/s
predict lospeed =  19.379560666666666 m/s
la誤差率 =  0.1255476670043382
lo誤差率 =  0.17708208034146014
finish
Current users number =  100
lat =  24.79204450588358
lon =  120.99987889649097
前移 2754668857093.7417 公尺
左移 13444417544068.717 公尺
real laspeed =  17.043069046575578 m/s
real lospeed =  23.615541574936625 m/s
predict laspeed =  20.031486666666666 m/s
predict lospeed =  19.363130666666667 m/s
la誤差率 =  0.12753956191570734
lo誤差率 =  0.17720152994640131
finish
Current users number =  100
lat =  24.792049566305746
lon =  120.99988525532376
前移 2754669419362.36 公尺
左移 13444418250604.984 公尺
real laspeed =  18.74230431429556 m/s
real lospeed =  23.551232562500378 m/s
predict laspeed =  19.970314666666663 m/s
predict lospeed =  19.386381333333333 m/s


Current users number =  100
lat =  24.792155477111788
lon =  121.00002554252356
前移 2754681187217.8774 公尺
左移 13444433838056.008 公尺
real laspeed =  16.989663115046586 m/s
real lospeed =  22.41300699602612 m/s
predict laspeed =  20.054985666666667 m/s
predict lospeed =  19.319241666666667 m/s
la誤差率 =  0.12459339394749286
lo誤差率 =  0.1775838141454653
finish
Current users number =  100
lat =  24.792160063233286
lon =  121.00003159335394
前移 2754681696786.423 公尺
左移 13444434510369.822 公尺
real laspeed =  16.985635179859553 m/s
real lospeed =  22.410482885064724 m/s
predict laspeed =  19.907028666666665 m/s
predict lospeed =  19.436733 m/s
la誤差率 =  0.12556071258368448
lo誤差率 =  0.1766677075729878
finish
Current users number =  100
lat =  24.792165063069103
lon =  121.00003824568627
前移 2754682252323.226 公尺
左移 13444435249517.186 公尺
real laspeed =  18.517910431396725 m/s
real lospeed =  24.638267893385127 m/s
predict laspeed =  19.972988666666666 m/s
predict lospeed =  19.454227666666664 m/s
la誤差率 = 

Current users number =  100
lat =  24.792269376188294
lon =  121.00017714743443
前移 2754693842658.161 公尺
左移 13444450683029.293 公尺
real laspeed =  16.79241050409372 m/s
real lospeed =  23.524001196367156 m/s
predict laspeed =  19.92126533333333 m/s
predict lospeed =  19.388727666666664 m/s
la誤差率 =  0.13021150091153125
lo誤差率 =  0.17520951759483724
finish
Current users number =  100
lat =  24.792274352426055
lon =  121.00018346426408
前移 2754694395572.9644 公尺
左移 13444451384898.549 公尺
real laspeed =  18.430510227450146 m/s
real lospeed =  23.395665388542746 m/s
predict laspeed =  20.022108 m/s
predict lospeed =  19.26913 m/s
la誤差率 =  0.1296107500134266
lo誤差率 =  0.17522555624198694
finish
Current users number =  100
lat =  24.792278849935187
lon =  121.0001902944733
前移 2754694895295.648 公尺
左移 13444452143809.982 公尺
real laspeed =  16.65744122718664 m/s
real lospeed =  25.29707117033737 m/s
predict laspeed =  19.948055333333333 m/s
predict lospeed =  19.421740999999997 m/s
la誤差率 =  0.1305287965

Current users number =  100
lat =  24.792387266970263
lon =  121.0003350918578
前移 2754706941620.8804 公尺
左移 13444468232392.111 公尺
real laspeed =  18.26378690094754 m/s
real lospeed =  23.30554599017921 m/s
predict laspeed =  20.057775 m/s
predict lospeed =  19.350519 m/s
la誤差率 =  0.13513421834962408
lo誤差率 =  0.174074273940341
finish
Current users number =  100
lat =  24.792391761854272
lon =  121.00034142672982
前移 2754707441051.8896 公尺
左移 13444468936266.08 公尺
real laspeed =  16.647718555136468 m/s
real lospeed =  23.462488932771656 m/s
predict laspeed =  19.963435 m/s
predict lospeed =  19.448778333333333 m/s
la誤差率 =  0.1357876387721233
lo誤差率 =  0.17404361052406694
finish
Current users number =  100
lat =  24.792396281157103
lon =  121.00034743865962
前移 2754707943196.149 公尺
左移 13444469604257.576 公尺
real laspeed =  16.738158631963216 m/s
real lospeed =  22.2664066608626 m/s
predict laspeed =  20.01436133333333 m/s
predict lospeed =  19.413952666666663 m/s
la誤差率 =  0.13639314306475778
lo誤